# Lembar Kerja UTS | MK Analisis Big Data Kelas A 
Fakultas Ilmu Komputer (FILKOM), Universitas Brawijaya (UB) 2020 | Dosen Pengampu: Imam Cholissodin, S.Si., M.Kom. 

Nama Mahasiswa: Muhammad Nafis | NIM: 175150200111008

*Kerjakan soal No. 1 sampai 10 pada (click) [Cell Utk Tempat Koding](#cell_utk_koding_)

# **PySpark**: Apache Spark Python API

### Set Konfigurasi utk konek ke Spark cluster, menggunakan SparkSession object dengan parameter berikut:

+ **appName:** set nama app yang akan ditampilkan pada [Spark Master Web UI](http://localhost:8080/);
+ **master:** set Spark Master URL, yang sudah terkoneksi dengan Spark Workers;
+ **spark.executor.memory:** set paling tidak <= dari set pada SPARK_WORKER_MEMORY config pada docker-compose.yml 

In [75]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-koding-uts").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

sc = spark.sparkContext

<a id='cell_utk_koding_'></a>

In [89]:
dataRDD = sc.textFile("data.csv") \
    .map(lambda line: line.split(";")) \
    .zipWithIndex().filter(lambda baris: baris[1] > 0) \
    .map(lambda x:x[0]) \
    .map(lambda x:x[1:])
print("DataRDD : ")
tampil = dataRDD.collect()
print(tampil)

isidataRDD = spark.createDataFrame(dataRDD).toDF("Fitur ke-1", "Fitur ke-2","Kelas")
isidataRDD.show()

total_dataTrain = isidataRDD.count()
FrekTiapKelasRDD = dataRDD.map(lambda x:(x[2],1.0)).reduceByKey(lambda x,y:x+y)
dataDFfrekKelas = spark.createDataFrame(FrekTiapKelasRDD).toDF("Kelas","Frek tiap kelas")
print("Data frek tiap kelas : ")
dataDFfrekKelas.show()

priorRDD = FrekTiapKelasRDD.map(lambda x:(x[0],x[1]/total_dataTrain))
priorDF = spark.createDataFrame(priorRDD).toDF("Kelas","Prior")
print("Prior tiap kelas : ")
priorDF.show()

dataTestRDD = sc.parallelize([('1','8.43','15.0')])
print("DataTest")
print(dataTestRDD.collect())
print("DataTestDF")
dataDF = spark.createDataFrame(dataTestRDD2)
dataDF.show()


byk_fitur = 2
fitur = []
kelas = ["good", "great"]
byk_kelas = len(kelas)
p_good = 1.
p_great = 1.
evi_good = 1.
evi_great = 1.
evidence = 1.

P_posterior = [1.,1.]
for m in range(0,1):
    oneDataTestRDD = dataTestRDD.zipWithIndex().filter(lambda baris:baris[1]==m).map(lambda x:x[0][1:3])
    print(oneDataTestRDD.collect())
    for i in range(0,byk_fitur):
        fitur.append(oneDataTestRDD.map(lambda x:x[i]).collect()[0])
        print("Fitur ke-"+str(i+1)+": "+ fitur[i])
        
        for j in range(0,byk_kelas):
            #menghitung mean dan standar deviasi untuk persiapan perhitungan likelihood
            mean = dataRDD.filter(lambda x:x[2]==kelas[j]).map(lambda x:float(x[i])).reduce(lambda x,y:x+y)/\
                    FrekTiapKelasRDD.filter(lambda x:x[0]==kelas[j]).map(lambda x:x[1]).collect()[0]
            frekKelas = FrekTiapKelasRDD.filter(lambda x:x[0]==kelas[j]).map(lambda x:x[1]).collect()[0]
            std = dataRDD.filter(lambda x:x[2]==kelas[j]).map(lambda x:((float(x[i])-mean)**2)/(frekKelas-1)).reduce(lambda x,y:x+y)**0.5
            
            #no 2, kode program menghitung nilai probabilitas likelihood
            myFnLikelihood = (lambda x:(1/( (2*(22/7)*(x[2])**2)**(0.5) ))*( 2.718**(-(((x[0]-x[1])**2)/(2*x[2]**2)))))
            contohDataRDD = sc.parallelize([(float(fitur[i]),mean,std)])
            likelihood = contohDataRDD.map(lambda x:myFnLikelihood(x)).collect()[0]
            print("Mean: ", mean)
            print("Standar Deviasi: ", std)
            print("P(",fitur[i],"|",kelas[j],")=",likelihood)
            
            P_posterior[j]*=likelihood
            
            print()
            #seleksi kondisi untuk persiapan perhitungan evidence
            if(kelas[j]=="Good"):
                p_good *= likelihood
                evi_good = p_good * float(priorRDD.filter(lambda x:x[0]==kelas[j]).map(lambda x:x[1]).collect()[0])
            else:
                p_great *= likelihood
                evi_great = p_great * float(priorRDD.filter(lambda x:x[0]==kelas[j]).map(lambda x:x[1]).collect()[0])
                
#  p(x) = [p(x1|good) * p(x2|good) * p(good)] + [p(x1|great) * p(x2|great) * p(great)]
evidence = evi_good + evi_great
print("Evidence: ", evidence)
P_posteriorRDD = sc.parallelize(P_posterior)
    
print()
print('Menampilkan hasil kelas dataTest')
for j in range(0,byk_kelas):
    P_posterior[j]*= priorRDD.filter(lambda x:x[0]==kelas[j]).map(lambda x:x[1]).collect()[0] / evidence
    print("Final P_posterior (",kelas[j],"| data Test ):", P_posterior[j])

print()

Argmax = P_posteriorRDD.zipWithIndex().sortBy(lambda x:x[0], ascending=False).map(lambda x:x[1]).collect()[0]    

print("Jadi data test : ", oneDataTestRDD.collect(),"masuk pada kelas :", kelas[Argmax])

DataRDD : 
[['8.0', '9.1', 'good'], ['7.8', '9.4', 'good'], ['12.5', '15.3', 'great'], ['13.4', '16.3', 'great'], ['12.1', '15.7', 'great']]
+----------+----------+-----+
|Fitur ke-1|Fitur ke-2|Kelas|
+----------+----------+-----+
|       8.0|       9.1| good|
|       7.8|       9.4| good|
|      12.5|      15.3|great|
|      13.4|      16.3|great|
|      12.1|      15.7|great|
+----------+----------+-----+

Data frek tiap kelas : 
+-----+---------------+
|Kelas|Frek tiap kelas|
+-----+---------------+
| good|            2.0|
|great|            3.0|
+-----+---------------+

Prior tiap kelas : 
+-----+-----+
|Kelas|Prior|
+-----+-----+
| good|  0.4|
|great|  0.6|
+-----+-----+

DataTest
[('1', '8.43', '15.0')]
DataTestDF
+---+----+----+
| _1|  _2|  _3|
+---+----+----+
|  1|8.43|15.0|
+---+----+----+

[('8.43', '15.0')]
Fitur ke-1: 8.43
Mean:  7.9
Standar Deviasi:  0.14142135623730964
P( 8.43 | good )= 0.0025164650581969887

Mean:  12.666666666666666
Standar Deviasi:  0.6658328118479396


In [91]:
lines = sc.parallelize([("ID-Twit-01","saya #Dia kamu #kalian #dia #dia dan #kalian adalah #saudara"),\
                        ("ID-Twit-02","#kamu #dia dan #SaYA serta kalian bersaudara")])

hashtags = lines.map(lambda x:x[1]).flatMap(lambda text: text.split(" "))
print(hashtags.collect())

toLowerCase = hashtags.map(lambda x:(x.lower(),1)).reduceByKey(lambda x,y:x+y).filter(lambda kata: '#' in kata[0])
print("\nHasil : ")
print(toLowerCase.collect())

['saya', '#Dia', 'kamu', '#kalian', '#dia', '#dia', 'dan', '#kalian', 'adalah', '#saudara', '#kamu', '#dia', 'dan', '#SaYA', 'serta', 'kalian', 'bersaudara']

Hasil : 
[('#saudara', 1), ('#saya', 1), ('#kamu', 1), ('#dia', 4), ('#kalian', 2)]


In [92]:
import math

data=[(1,'aku cinta Indonesia'),(2,"Aku anti Perpecahan dan aku anti permusuhan"), \
      (3,"Pancasila adalah dasar negara indonesia")]
lines=sc.parallelize(data)

hapusword = ['dan', 'Adalah']
hapusword_lower = [x.lower() for x in hapusword]

# ((x[0],i),1) merupakan kombinasi ("id dokumen", "term", dan set nilai 1 utk tiap term)
map1=lines.flatMap(lambda x: [((x[0],i),1) for i in x[1].split() if i.lower() not in hapusword_lower])
reduce=map1.map(lambda term:((term[0][0],term[0][1].lower()),term[1])).reduceByKey(lambda x,y:x+y)
tf=reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))

map3=reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
map3.collect()
map4=map3.map(lambda x:(x[0],x[1][2]))
map4.collect()
reduce2=map4.reduceByKey(lambda x,y:x+y)
reduce2.collect()
idf=reduce2.map(lambda x: (x[0],math.log10(len(data)/x[1])))
idf.collect()

rdd=tf.join(idf)
rdd=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()
rdd.collect()
rdd=rdd.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))

rdd.toDF(["DocumentId","Token","TF","IDF","TF-IDF"]).show()

rddDistinct = rdd.map(lambda x:x[1]).distinct()
# rddDistinct = rdd.map(lambda x:x[1])
byk_fitur = rddDistinct.count()
print(rddDistinct.collect())

nilaiMaks = rdd.map(lambda x:(x[0],x[4])).reduceByKey(lambda x,y:max(x,y))
print(nilaiMaks.collect())

print("Data TF-IDF terbesar :")
makstoDF = spark.createDataFrame(nilaiMaks).toDF("Data id","Nilai Terbesar")
makstoDF.show()

+----------+----------+---+-------------------+-------------------+
|DocumentId|     Token| TF|                IDF|             TF-IDF|
+----------+----------+---+-------------------+-------------------+
|         1|     cinta|  1|0.47712125471966244|0.47712125471966244|
|         1|       aku|  1|0.17609125905568124|0.17609125905568124|
|         1| indonesia|  1|0.17609125905568124|0.17609125905568124|
|         2|       aku|  2|0.17609125905568124| 0.3521825181113625|
|         2|permusuhan|  1|0.47712125471966244|0.47712125471966244|
|         2|      anti|  2|0.47712125471966244| 0.9542425094393249|
|         2|perpecahan|  1|0.47712125471966244|0.47712125471966244|
|         3|     dasar|  1|0.47712125471966244|0.47712125471966244|
|         3| pancasila|  1|0.47712125471966244|0.47712125471966244|
|         3| indonesia|  1|0.17609125905568124|0.17609125905568124|
|         3|    negara|  1|0.47712125471966244|0.47712125471966244|
+----------+----------+---+-------------------+-

In [95]:
# https://drive.google.com/file/d/1YVHgU7TXgFc_u6tMqQWJiXBR4Yf4_NQk/view?usp=sharing
#install wget terlebih dahulu
!echo y | apt-get install wget

from timeit import default_timer as timer
from datetime import timedelta

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YVHgU7TXgFc_u6tMqQWJiXBR4Yf4_NQk' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1YVHgU7TXgFc_u6tMqQWJiXBR4Yf4_NQk" -O 'tweet.csv' && rm -rf /tmp/cookies.txt

nama_header = ['created_at', 'in_reply_to_status_id', 'id_str', \
 'retweeted', 'in_reply_to_user_id_str', 'coordinates', \
 'retweet_count', 'contributors', 'favorite_count', \
 'favorited', 'in_reply_to_status_id_str', 'source', \
 'in_reply_to_user_id', 'user_id_str', 'user_screen_name', \
 'place', 'geo', 'text']

byk_kolom= len(nama_header)
print(byk_kolom)

# Load Dataset
start = timer()
dataRDD = sc.textFile("tweet.csv") \
    .filter(lambda line: line.count(",") == byk_kolom-1 ) \
    .map(lambda line: line.split(",")) \
    .zipWithIndex().filter(lambda baris: baris[1] > 0) \
    .map(lambda x:x[0])

print("Menampilkan sebagian dataRDD:")
print(dataRDD.take(7))
print()
print("Banyak baris pada data tweet.csv = ",dataRDD.count()," baris data twitter tanpa malform")
end = timer()
print("Waktu load data dan penghitungan banyak data = ", end - start, " detik")
#print(timedelta(seconds=end-start))

#Menampilkan 10 teratas tweet favorit
dataRDD_fav = dataRDD.map(lambda x:(x[14],1)).reduceByKey(lambda x,y:x+y).sortBy(lambda x:x[1],ascending=False)
print("Menampilkan data username dan jumlahnya")
data_favtoDF = spark.createDataFrame(dataRDD_fav).toDF("Name","Jumlah")
data_favtoDF.show()
print("Menampilkan 10 teratas tweet favorit")
dataRDD_fav10 = dataRDD_fav.take(10)
dataRDD_fav10DF = spark.createDataFrame(dataRDD_fav10).toDF("Name","jumlah")
dataRDD_fav10DF.show()

temp = sc.parallelize([(4,2), (8,1), (6,4), (2,3)])
sort1 = temp.sortBy(lambda x:x[1], ascending=False)
dataDF = spark.createDataFrame(sort1)
dataDF.show()

jumlahTwet = dataRDD.map(lambda x:(x[17], 1)).reduceByKey(lambda x,y:x+y).sortBy(lambda x:x[1], ascending=False)

print("Menampilkan tweet terbanyak:")
dataDF = spark.createDataFrame(jumlahTwet).toDF("Tweet", "Jumlah")
dataDF.show()

print("Menampilkan 2 tweet terbanyak:")
jumlahTwet2 = jumlahTwet.take(2)
print(jumlahTwet2)
dataDF = spark.createDataFrame(jumlahTwet2).toDF("Tweet", "Jumlah")
dataDF.show()

jumlahTwet = dataRDD.map(lambda x:(x[17], 1)).reduceByKey(lambda x,y:x+y).sortBy(lambda x:x[1], ascending=False)
print("Menampilkan tweet terbanyak:")
dataDF = spark.createDataFrame(jumlahTwet).toDF("Tweet", "Jumlah")
dataDF.show()
print("Menampilkan 2 tweet terbanyak:")
jumlahTwet2 = jumlahTwet.take(2)
dataDF = spark.createDataFrame(jumlahTwet2).toDF("Tweet", "Jumlah")
dataDF.show()

dataRDD_text_plg_favorit_alt2 = dataRDD.map(lambda x:((x[17],x[9]=='True'),1)).reduceByKey(lambda x,y:x+y)\
.sortBy(lambda x:x,ascending=False).take(5)
print(dataRDD_text_plg_favorit_alt2)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.20.1-1.1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
--2020-11-18 02:39:29--  https://docs.google.com/uc?export=download&confirm=9Bp8&id=1YVHgU7TXgFc_u6tMqQWJiXBR4Yf4_NQk
Resolving docs.google.com (docs.google.com)... 172.217.9.206, 2607:f8b0:4004:811::200e
Connecting to docs.google.com (docs.google.com)|172.217.9.206|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-3g-docs.googleusercontent.com/docs/securesc/fk6kv7a9pnosa987t9j5c6u65vcjj3f6/7ven6sc61dvc4malkujamrvapg1mueqb/1605667125000/09783780208665738920/17838546180223045918Z/1YVHgU7TXgFc_u6tMqQWJiXBR4Yf4_NQk?e=download [following]
--2020-11-18 02:39:29--  https://doc-0s-3g-docs.googleusercontent.com/docs/securesc/fk6kv7a9pnosa987t9j5c6u65vcjj3f6/7ven6sc61dvc4malkujamrvapg1mueqb/1605667125000/09783780208665738920/1783854618

In [7]:
# =========================

# Code Petunjuk **koding PySpark untuk UTS**: 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.20.1-1.1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [15]:
# utk download data tweet.csv dari link public G Drive
# https://drive.google.com/file/d/1YVHgU7TXgFc_u6tMqQWJiXBR4Yf4_NQk/view?usp=sharing

# lakukan instalasi dulu wget dgn "!echo y | apt-get install wget"
#!echo y | apt-get install wget

# cara download-nya sbg berikut:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YVHgU7TXgFc_u6tMqQWJiXBR4Yf4_NQk' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1YVHgU7TXgFc_u6tMqQWJiXBR4Yf4_NQk" -O 'tweet.csv' && rm -rf /tmp/cookies.txt

--2020-11-14 15:42:21--  https://docs.google.com/uc?export=download&confirm=sCHY&id=1YVHgU7TXgFc_u6tMqQWJiXBR4Yf4_NQk
Resolving docs.google.com (docs.google.com)... 172.217.8.14, 2607:f8b0:4004:811::200e
Connecting to docs.google.com (docs.google.com)|172.217.8.14|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-4g-docs.googleusercontent.com/docs/securesc/qnmh9no26unvrnju3jeo5tqv1m9i455h/ca1b6j08m8ft9r09ui4cjauplegio6k5/1605368475000/09783780208665738920/10219236717386092432Z/1YVHgU7TXgFc_u6tMqQWJiXBR4Yf4_NQk?e=download [following]
--2020-11-14 15:42:21--  https://doc-0g-4g-docs.googleusercontent.com/docs/securesc/qnmh9no26unvrnju3jeo5tqv1m9i455h/ca1b6j08m8ft9r09ui4cjauplegio6k5/1605368475000/09783780208665738920/10219236717386092432Z/1YVHgU7TXgFc_u6tMqQWJiXBR4Yf4_NQk?e=download
Resolving doc-0g-4g-docs.googleusercontent.com (doc-0g-4g-docs.googleusercontent.com)... 172.217.13.65, 2607:f8b0:4004:807::2001
Connecting to doc-0g-4g-

In [16]:
#cek ukuran file "tweet.csv"
!ls ./tw* -a -l --block-size=m

-rw-r--r-- 1 root root 1218M Nov 14 15:43 ./tweet.csv


18


Menampilkan sebagian dataRDD:
[['2019-07-22 05:41:47', '1.1531772532341473e+18', '1153178304603545600', 'False', '731167584.0', '', '0', '', '0', 'False', '1.1531772532341473e+18', 'Twitter for iPhone', '731167584.0', '1003356794683416576', 'CatsRule98', '', '', '@RonBrownstein That’s why #IStandWithIlhan'], ['2019-07-22 05:41:48', '', '1153178308776869888', 'False', '', '', '0', '', '0', 'False', '', 'Twitter for Android', '', '1097334060', 'JejakaShahid', '', '', 'RT @mathsanova: mana pakai faceapp lagi 🤪🤣 https://t.co/pO4adR5dI6'], ['2019-07-22 05:41:49', '', '1153178311515766785', 'False', '', '', '0', '', '0', 'False', '', 'Twitter for Android', '', '1014117841', 'nurlailaeksa', '', '', '"RT @BadmintonTalk: #BtalkBWFRankPrediction MS after #BlibliIndonesiaOpen2019 and #RussianOpenSuper100 '], ['2019-07-22 05:41:49', '', '1153178313948426241', 'False', '', '', '0', '', '0', 'False', '', 'Twitter for Android', '', '701611356496113664', 'sticvh', '', '', 'RT @Husen_Jafar: FaceApp har

Menampilkan data username dan jumlahnya
+---------------+------+
|           Name|Jumlah|
+---------------+------+
|     wordnuvola|  2208|
| GooglePayIndia|  2112|
|       Weinbach|  1808|
|       WwuRadio|  1179|
|        pogosj1|  1158|
|Christo73106853|  1112|
|lililin79416363|   933|
|MercadoLechugas|   917|
|mars_project_on|   791|
|     dpdcentral|   704|
|         56_aae|   681|
|    iamtaquirah|   677|
|     Sravslucky|   669|
|    RBitterwolf|   656|
|      4oesmomma|   642|
|     xxzzxxzzzz|   641|
|   SteveSpeedZA|   638|
|  jeffdavisshow|   635|
|     AqsaEshaal|   610|
|   DinaElHakim4|   608|
+---------------+------+
only showing top 20 rows

Manmpilkan 10 teratas tweet favorit
+---------------+------+
|           Name|jumlah|
+---------------+------+
|     wordnuvola|  2208|
| GooglePayIndia|  2112|
|       Weinbach|  1808|
|       WwuRadio|  1179|
|        pogosj1|  1158|
|Christo73106853|  1112|
|lililin79416363|   933|
|MercadoLechugas|   917|
|mars_project_on|   791

In [20]:
# Buatlah kode program Spark RDD atau Spark SQL untuk mendapatkan 10 “text” 
# yaitu Tweet apa yang paling difavoritkan dan user_screen_name-nya?
# dataRDD_text_plg_favorit_alt1 = dataRDD.map(lambda x:(x[17],int(x[8]))).reduceByKey(lambda x,y:x+y)\
# .sortBy(lambda x:x[1],ascending=False).take(5)
# print(dataRDD_text_plg_favorit_alt1)

+---+---+
| _1| _2|
+---+---+
|  6|  4|
|  2|  3|
|  4|  2|
|  8|  1|
+---+---+



Menampilkan tweet terbanyak:
+--------------------+------+
|               Tweet|Jumlah|
+--------------------+------+
|RT @LiamPayne: 9 ...|133763|
|RT @Zayg23: I cra...|131023|
|RT @BTS_twt: Seou...| 54910|
|"RT @BTS_twt: 😱😱💜| 54180|
|RT @realDonaldTru...| 19948|
|RT @TheXFactor: W...| 16612|
|RT @LeftAtLondon:...| 16298|
|"RT @BigHitEnt: S...| 14825|
|RT @JoshDevineDru...| 14374|
|"RT @hendralm: Se...| 14233|
|RT @PhilPrajya: ร...| 14006|
|"RT @Abdulaziz_Hm...| 13333|
|RT @hellofeb22: ถ...| 13279|
|"RT @NCTsmtown_DR...| 12026|
|RT @Lazy_and_lame...| 10438|
|"RT @Pantene: I G...| 10437|
|RT @aserembaX: Na...|  9476|
|RT @chartdata: .@...|  8936|
|"RT @Believe_In_L...|  8228|
|RT @mistxxke: for...|  8096|
+--------------------+------+
only showing top 20 rows

Menampilkan 2 tweet terbanyak:
[('RT @LiamPayne: 9 years... Amazing to look back at all the memories and even now you got us trending worldwide after 9 years. You truly are…', 133763), ('RT @Zayg23: I cracked an egg and it was d

Menampilkan tweet terbanyak:
+--------------------+------+
|               Tweet|Jumlah|
+--------------------+------+
|RT @LiamPayne: 9 ...|133763|
|RT @Zayg23: I cra...|131023|
|RT @BTS_twt: Seou...| 54910|
|"RT @BTS_twt: 😱😱💜| 54180|
|RT @realDonaldTru...| 19948|
|RT @TheXFactor: W...| 16612|
|RT @LeftAtLondon:...| 16298|
|"RT @BigHitEnt: S...| 14825|
|RT @JoshDevineDru...| 14374|
|"RT @hendralm: Se...| 14233|
|RT @PhilPrajya: ร...| 14006|
|"RT @Abdulaziz_Hm...| 13333|
|RT @hellofeb22: ถ...| 13279|
|"RT @NCTsmtown_DR...| 12026|
|RT @Lazy_and_lame...| 10438|
|"RT @Pantene: I G...| 10437|
|RT @aserembaX: Na...|  9476|
|RT @chartdata: .@...|  8936|
|"RT @Believe_In_L...|  8228|
|RT @mistxxke: for...|  8096|
+--------------------+------+
only showing top 20 rows

Menampilkan 2 tweet terbanyak:
+--------------------+------+
|               Tweet|Jumlah|
+--------------------+------+
|RT @LiamPayne: 9 ...|133763|
|RT @Zayg23: I cra...|131023|
+--------------------+------+



[(('🧸💞 https://t.co/2Fxhd6GSSo', False), 1), (('🧸💕 https://t.co/xhnYD7ZAMW', False), 1), (('🧸ARMY are extremely dedicated and think #BTS is top tier! We really want to hear #SeoulTownRoad by @LilNasX feat RM… https://t.co/dAxEW2CCDm', False), 1), (('🧸 https://t.co/vZUSTP24D7', False), 1), (('🧸 https://t.co/rvTZSnE4OE', False), 1)]


In [22]:
# Ref:
# [1] Cholissodin, I., Riyandani, E., 2016, Analisis Big Data, Fakultas Ilmu Komputer, Universitas Brawijaya, Malang.
# [2] Cholissodin, I. and Supianto, A. A. 2019. Enhancement Full Open Source Hadoop Distribution Universal Big Data Up Projects (UBig) From Education To Enterprise. 2019 International Conference on Sustainable Information Engineering and Technology (SIET), Lombok, Indonesia, 2019, pp. 90-93, doi: 10.1109/SIET48054.2019.8986040.
# [3] Miao, K., Li, J., Hong, W. & Chen, M. 2020. A Microservice-Based Big Data Analysis Platform for Online Educational Applications. Annual Review of Anthropology, 2020, [ "6929750" ]. Available from: https://doi.org/10.1146/annurev.anthro.33.070203.144008
# [4] Roy, S., et al. 2017. IoT, big data science & analytics, cloud computing and mobile app based hybrid system for smart agriculture. 2017 8th Annual Industrial Automation and Electromechanical Engineering Conference (IEMECON), Bangkok, 2017, pp. 303-304, doi: 10.1109/IEMECON.2017.8079610.
# [5] Dabek, F. 2016. Leveraging Big Data to Provide a Web Service That Provides the Likelihood of Developing Psychological Conditions after a Concussion. 2016 IEEE International Conference on Mobile Services (MS), San Francisco, CA, 2016, pp. 160-165, doi: 10.1109/MobServ.2016.32.
# [6] Andy A. 2016. Cloud Computing Part 5: SaaS (Software as a Service). Available at: https://andypi.co.uk/2016/05/23/cloud-computing-part-5-saas-software-as-a-service/ (Accessed on July 30, 2020).
# [7] Putra, N. A., Putri, A. T., Prabowo, D. A., Surtiningsih, L., Arniantya, R., Cholissodin, I. 2017. Klasifikasi Sepeda Motor Berdasarkan Karakteristik Konsumen Dengan Metode K-Nearest Neighbour Pada Big Data Menggunakan Hadoop Single Node Cluster. Jurnal Teknologi Informasi dan Ilmu Komputer (JTIIK) FILKOM UB Vol. 4 No. 2, 81-86.
# [8] Naveen, N. 2020. Hadoop MapReduce – Key Features & Highlights. Available at: https://intellipaat.com/blog/tutorial/big-data-and-hadoop-tutorial/hadoop-mapreduce-key-features-highlights/ (Accessed on March 8, 2020).
# [9] stackchief. 2017. MapReduce Quick Explanation. Available at: https://www.stackchief.com/blog/MapReduce%20Quick%20Explanation (Accessed on March 8, 2020).
# [10] ..